In [1]:
## Placeholder for a script that will download data from flywheel and save it to a local directory
import os
import flywheel
import flywheel.api.acquisitions_api
import flywheel.finder
#import flywheel.flywheel
from pathlib import Path
#import logging
from getpass import getpass
import datetime
from time import sleep
import platform
import zipfile

In [3]:
#Specify the root project path here
root_data = "C:\\Users\\Charm\\projects\\test_folder"
reserved_characters = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
#Create an instance of AcquisitionsApi
AcquisitionsApi = flywheel.api.acquisitions_api.AcquisitionsApi()
#Enable long path names for Windows with Powershell Command.
#if os.system == 'Windows':
    #os.system('New-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\FileSystem" \' -Name "LongPathsEnabled" -Value 1 -PropertyType DWORD -Force')
    #os.system('Set-ItemProperty -Path HKLM:SYSTEM\CurrentControlSet\Control\FileSystem -Name LongPathsEnabled -Value 1')
#else:
    #pass
    
    


In [ ]:
#Have the user enter their API key
api_key = getpass('Please enter your API key: ')
#Initialize the client 
fw = flywheel.Client(api_key)
#Delete the API key
del api_key

In [40]:
#Create an instance of Finder
Finder = flywheel.finder.Finder(fw, 'iter_find')

In [29]:
os.path.exists(root_data)

True

In [105]:
#new_acquisitions_list = fw.acquisitions.find('created>2024-05-15')
# 'created>2024-05-15T00:00:00.000Z'
#test = fw.acquisitions.find('created>2024-05-15T00:00:00.000Z')

query = 'file.created>=2024-05-15T00:00:00.000Z AND project.label= fus'
#Size references the number of results that you'd like to return.
new_files_list =fw.search({'structured-query': query, 'return_type': 'file'}, size = 10000)
print(len(new_files_list))
#for new_file in new_files_list:
    
print(new_files_list[0])


5052
{'acquisition': None,
 'analysis': None,
 'collection': None,
 'file': {'classification': {},
          'created': datetime.datetime(2022, 10, 14, 19, 54, 46, 158000, tzinfo=tzutc()),
          'name': '1.3.12.2.1107.5.2.43.166037.2022101415511175483519909.0.0.0.dicom.zip',
          'size': 3172170,
          'type': 'dicom'},
 'group': {'id': 'bikson', 'label': 'Bikson'},
 'parent': {'analyses': None,
            'collections': None,
            'copy_of': None,
            'created': None,
            'files': None,
            'id': '6349be86244436ef4c8fa3fb',
            'info': None,
            'info_exists': None,
            'join_origin': None,
            'label': None,
            'metadata': None,
            'modified': None,
            'notes': None,
            'original_copy_of': None,
            'parents': None,
            'permissions': None,
            'revision': 1,
            'session': None,
            'tags': None,
            'timestamp': None,
     

In [68]:
date = '2024-06-04'
time_stamp = '00:00:00'

#Returns a list of dictionaries.
file_ids = []
path = []
#Returns a list of dictionaries, will get index out of range error if the list is empty.
new_acquisitions_list = fw.acquisitions.find('created>2024-05-15')
#List of dictionaries
print(len(new_acquisitions_list))
#Get all files in one acquisition
#new_files = new_acquisitions_list[0]['files']
#for file in new_files:
    #file_ids.append(file['file_id'])
#For all the acquisitions in new list and for every file in the acquisition, get the path of the file.
if len(new_acquisitions_list) <= 20:
    for i in range(len(new_acquisitions_list)):
        for j in range(len(new_acquisitions_list[i]['files'])):
            new_project = fw.get_project(new_acquisitions_list[i]['files'][j]['parents']['project'])
            new_subject = fw.get_subject(new_acquisitions_list[i]['files'][j]['parents']['subject'])
            new_session = fw.get_session(new_acquisitions_list[i]['files'][j]['parents']['session'])
            new_acquisition = fw.get_acquisition(new_acquisitions_list[i]['files'][j]['parents']['acquisition'])
            path.append(f'{root_data}/{new_acquisitions_list[i]["files"][j]["parents"]["group"]}/{new_project}/{new_subject}/{new_session}/{new_acquisition}')
else:
    for i in range(20):
        for j in range(len(new_acquisitions_list[i]['files'])):
            new_project = fw.get_project(new_acquisitions_list[i]['files'][j]['parents']['project'])
            new_subject = fw.get_subject(new_acquisitions_list[i]['files'][j]['parents']['subject'])
            new_session = fw.get_session(new_acquisitions_list[i]['files'][j]['parents']['session'])
            new_acquisition = fw.get_acquisition(new_acquisitions_list[i]['files'][j]['parents']['acquisition'])
            path.append(f'{root_data}/{new_acquisitions_list[i]["files"][j]["parents"]["group"]}/{new_project}/{new_subject}/{new_session}/{new_acquisition}')
print(path)
    
#print(new_acquisitions[0]['files'])
#print(first_new_acq['file_id'])
    
def check_for_updates():
    global date
    global time_stamp
    new_files_list =fw.search({'structured-query': query, 'return_type': 'file'}, size = 10000)
    for new_acquisition in new_acquisitions_list:
        get_file(new_acquisition['file_id'])
        
    #If the collection is not found, update the date and time_stamp.
    if not new_collections:
        #The format must be year-month-day, date.today() returns the date in the format 'year-month-day'.
        date = datetime.date.today()
        #The format must be hour:minute:second, datetime.now().strftime('%H:%M:%S') returns the time in the format 'hour:minute:second'.
        time_stamp = datetime.now().strftime('%H:%M:%S')
    else:
        for session in new_collections.sessions.iter():
            #Get the project the collection is under, session.project is the project id.
            project = fw.get_project(session.project)
            for subject in project.subjects.iter():
                subject = fw.get_subject()
                for acquisition in subject.acquisitions.iter():
                    for file in acquisition.files.iter():
                        collection = acquisition.id
            download_recent_flywheel_data(fw, collection, date)
            #The format must be year-month-day, date.today() returns the date in the format 'year-month-day'.
            date = datetime.date.today()
            #The format must be hour:minute:second, datetime.now().strftime('%H:%M:%S') returns the time in the format 'hour:minute:second'.
            time_stamp = datetime.now().strftime('%H:%M:%S')
    sleep(60)
    return date, time_stamp

#check_for_updates()

0
[]


In [ ]:
#Goals: 1.) Automatically update the local directory with flywheel data in real time. 2.) Download the data to the local directory, including handling any duplicate files, incomplete resolver_path, or additional artifacts such as jobs and analyses.
test_resolver_path = 'fw://group_id/project_id/subject_id/session_id/acquisition_id/files/file_id'
#Goal download flywheel_data to the local directory. Meant to be kept running and update in real time that way it doesn't check the same files over and over again.
#Function downloads all data from specified group(s) to the local directory one file at a time (down up approach).
def download_recent_flywheel_data(fw, resolver_path, time_stamp):
    global date
    global time_stamp
    #Open the file where all the flywheel data will be 
    #os.open('C:\\Users\\Charm\\projects')
    #Remove the prefix from the resolver path
    resolver_path = resolver_path.removeprefix('fw://')
    #If the resolver path does not exist, create the path in the local directory.
    if not os.path.exists(resolver_path):
        # Turn the resolver path into a list s.t. [group_id, project_id, subject_id, session_id, acquisition_id, files, file_id]
        resolver_path_list = resolver_path.split('/')
        #Iterate through the resolver path list and check if, for each level, the folder exists in the local directory, if it doesn't, create it. If the path leads to a file, download it in its respective folder.
        for path in resolver_path_list:
            if 'files' not in path:
                if path == resolver_path_list[0]:
                    updating_resolver_path = path
                else:
                    updating_resolver_path = '/'.join(updating_resolver_path, path)
                #Can also do Path(path).exists() instead of os.path.exists(path)
                if os.path.exists(path) == False:
                    os.makedirs(path)
        #If the path contains a file, check if the path exists, if it doesn't add the file to the path. If it does, create a duplicate file with the same name? *Note to self: See how flywheel handles duplicate files*
            else:
                #If the file does not exist, download it to the local directory.
                if os.path.exists(path) == False:
                    file = fw.get_container_file(path)
                    acquisition.download_file(file.name, file.name)
                #If the file exists, create a duplicate file with a different name.
                else:
                    #Create a duplicate file with a different name.
                    print('There is a file with this name already in the local directory. A duplicate file will be created with a different name.')
                    file = fw.get_container_file(path)
                    acquisition.download_file(file.name, file.name + '_duplicate')

    


In [ ]:
#Function downloads all data from specified group(s) to the local directory group by group (up down approach).
#*Note to Self: .lookup takes in the labels as input, .get takes in the ID as input.
def download_all_flywheel_data():
    if platform.system() != 'Windows':
        group_list = []
        project_list = []
        #For each group in the flywheel instance, check if the user would like to download all the data from the group, if yes, add the group to the group list.
        for group in fw.groups():
            user_group = input(f'Would you like to download all the data from the group {group.label}? (yes/no): ')
            if user_group == 'yes':
                group_list.append(group)
            elif user_group == 'no':
                continue
        for group in group_list:
            new_group_label = group.label
            for character in group.label: 
                if character in reserved_characters: 
                    new_group_label = new_group_label.replace(character, '_') 
            #Default value of exist_ok is False and will raise an OSError if the directory exists when using os.makedirs. If using Path.mkdir, the default value of exist_ok is False and will raise a FileExistsError if the directory exists. Path.mkdir takes in path, os.makedirs takes in strings.
            #Create a directory for the group in the local directory.
            try:
                os.makedirs(root_data + '/' + new_group_label)
            except OSError:
                pass
            #For each project in the group, check if the user would like to download all the data from the project, if yes, add the project to the project list.
            for project in group.projects():
                user_project = input(f'Would you like to download all the data from the project {project.label}? (yes/no): ')
                if user_project == 'yes':
                    project_list.append(project)
                elif user_project == 'no':
                    continue
            #Create a directory for the project in the local directory.
            for project in project_list:
                new_project_label = project.label
                for character in project.label: 
                    if character in reserved_characters: 
                        new_project_label = new_project_label.replace(character, '_')
                try:
                    os.makedirs(root_data + '/' + new_group_label + '/' + new_project_label)
                except OSError:
                    pass
                #Create a directory for the subject in the local directory.
                for subject in project.subjects():
                    new_subject_label = subject.label
                    for character in subject.label: 
                        if character in reserved_characters: 
                            new_subject_label = new_subject_label.replace(character, '_')
                    try:
                        os.makedirs(root_data + '/' + new_group_label + '/' + new_project_label + '/' + new_subject_label)
                    except OSError:
                        pass
                    #Create a directory for the session in the local directory.
                    for session in subject.sessions():
                        new_session_label = session.label
                        for character in session.label: 
                            if character in reserved_characters: 
                                new_session_label = new_session_label.replace(character, '_')
                        try:
                            os.makedirs(root_data + '/' + new_group_label + '/' + new_project_label + '/' + new_subject_label + '/' + new_session_label)
                        except OSError:
                            pass
                        for acquisition in session.acquisitions():
                            new_acquisition_label = acquisition.label
                            for character in acquisition.label:
                                if character in reserved_characters:
                                    new_acquisition_label = new_acquisition_label.replace(character, '_')
                            try:
                                os.makedirs(root_data + '/' + new_group_label + '/' + new_project_label + '/' + new_subject_label + '/' + new_session_label + '/' + new_acquisition_label)
                            except OSError:
                                pass
                            #For each file in the acquisition, check if the file exists in the local directory, if it doesn't, download it.
                            for file in acquisition.files:
                                new_file_name = file.name
                                acquisition = fw.get_acquisition(acquisition.id)
                                for character in file.name:
                                    if character in reserved_characters:
                                        new_file_name = new_file_name.replace(character, '_')
                                        
                                path = root_data + '/' + new_group_label + '/' + new_project_label + '/' + new_subject_label + '/' + new_session_label + '/' + new_acquisition_label
                                
                                if file not in os.walk(path):   
                                    acquisition.download_file(file.name, path + '/' + file.name)
                                    #AcquisitionsApi.download_file_from_acquisition(acquisition_id = acquisition.id, file_name = file.name, dest_file = f'{path}/{file.name}')
                                        #file.download(f'{path}/{file.name}') 
                                else:
                                    #Create a duplicate file with a different name.
                                    print(f'There is a file with the name {file.name} already in the local directory. No action.')
    else:
        download_all_flywheel_data_windows()
    
download_all_flywheel_data()


In [ ]:
def download_all_flywheel_data_windows():
    if platform.system() == 'Windows':
        group_list = []
        project_list = []
        #For each group in the flywheel instance, check if the user would like to download all the data from the group, if yes, add the group to the group list.
        for group in fw.groups():
            user_group = input(f'Would you like to download all the data from the group {group.label}? (yes/no): ')
            if user_group == 'yes':
                group_list.append(group)
            elif user_group == 'no':
                continue
        for group in group_list:
            new_group_label = group.label
            for character in group.label: 
                if character in reserved_characters: 
                    new_group_label = new_group_label.replace(character, '_') 
            #Default value of exist_ok is False and will raise an OSError if the directory exists when using os.makedirs. If using Path.mkdir, the default value of exist_ok is False and will raise a FileExistsError if the directory exists. Path.mkdir takes in path, os.makedirs takes in strings.
            #Create a directory for the group in the local directory.
            try:
                os.makedirs(root_data + '\\' + new_group_label)
            except OSError:
                pass
            #For each project in the group, check if the user would like to download all the data from the project, if yes, add the project to the project list.
            for project in group.projects():
                user_project = input(f'Would you like to download all the data from the project {project.label}? (yes/no): ')
                if user_project == 'yes':
                    project_list.append(project)
                elif user_project == 'no':
                    continue
            #Create a directory for the project in the local directory.
            for project in project_list:
                new_project_label = project.label
                for character in project.label: 
                    if character in reserved_characters: 
                        new_project_label = new_project_label.replace(character, '_')
                try:
                    os.makedirs(root_data + '\\' + new_group_label + '\\' + new_project_label)
                except OSError:
                    pass
                #Create a directory for the subject in the local directory.
                for subject in project.subjects():
                    new_subject_label = subject.label
                    for character in subject.label: 
                        if character in reserved_characters: 
                            new_subject_label = new_subject_label.replace(character, '_')
                    try:
                        os.makedirs(root_data + '\\' + new_group_label + '\\' + new_project_label + '\\' + new_subject_label)
                    except OSError:
                        pass
                    #Create a directory for the session in the local directory.
                    for session in subject.sessions():
                        new_session_label = session.label
                        for character in session.label: 
                            if character in reserved_characters: 
                                new_session_label = new_session_label.replace(character, '_')
                        try:
                            os.makedirs(root_data + '\\' + new_group_label + '\\' + new_project_label + '\\' + new_subject_label + '\\' + new_session_label)
                        except OSError:
                            pass
                        for acquisition in session.acquisitions():
                            new_acquisition_label = acquisition.label
                            for character in acquisition.label:
                                if character in reserved_characters:
                                    new_acquisition_label = new_acquisition_label.replace(character, '_')
                            try:
                                os.makedirs(root_data + '\\' + new_group_label + '\\' + new_project_label + '\\' + new_subject_label + '\\' + new_session_label + '\\' + new_acquisition_label)
                            except OSError:
                                pass
                            #For each file in the acquisition, check if the file exists in the local directory, if it doesn't, download it.
                            for file in acquisition.files:
                                new_file_name = file.name
                                acquisition = fw.get_acquisition(acquisition.id)
                                for character in file.name:
                                    if character in reserved_characters:
                                        new_file_name = new_file_name.replace(character, '_')
                                path = root_data + '\\' + new_group_label + '\\' + new_project_label + '\\' + new_subject_label + '\\' + new_session_label + '\\' + new_acquisition_label
                                if file not in os.walk(path):
                                    acquisition.download_file(file.name, path + '\\' + file.name)
                                else:
                                    #Create a duplicate file with a different name.
                                    print(f'There is a file with the name {file.name} already in the local directory. No action.')
    else:
        download_all_flywheel_data()


In [ ]:
## The data will then be ready to be pre-processed

In [ ]:
## enter PommieBoss
folder = "C:\\Users\\Charm\\projects\\test_folder\\Bikson\\ThermFlow_Rollers\\MB_thermtest\\unknown\\localizer"

# get the first .zip file in folder
zipfiles = [f for f in os.listdir(folder) if f.endswith('.zip')]

import zipfile

with zipfile.ZipFile(os.path.join(folder, zipfiles[0]), 'r') as zip_ref:
    # extract all files to folder
    zip_ref.extractall(folder)

In [ ]:
zipfile = "1.3.12.2.1107.5.2.43.166037.2022101415453511757218637.0.0.0.dicom"
full_zipfile = os.path.join(folder, zipfile)
# check if full_zipfile exists
if os.path.exists(full_zipfile):
    # unzip the file
    subprocess.run(["7z", "x", full_zipfile, f"-o{folder}"])

In [ ]:
os.path.exists(full_zipfile)

In [ ]:
!dir C:\Users\Charm\projects\test_folder\Bikson\ThermFlow_Rollers\MB_thermtest\unknown\localizer

In [ ]:
#if file.zip_member_count is not None:
                                print(file)
                                #Obtain the acquisition object.
                                #Print the zip file info.
                                #print(acq.get_aquisition_file_zip_info(acquisition.id, file.name))
                                #for member in acquisition.get_file_zip_info(file):
#with zipfile.ZipFile(file.name, 'r') as zip_file:
                                    #For each file in the zip file, download the file.
                                        #zip_file.extractall()
                                        #print(file_list)
                                    #data = file.read(name)
                                    #info.download(f'{path}/{file.name}')
                                    #member.download(f'{path}/{member.name}')
                                        #for i in range(len(file_list)):
                                            #file_list[i].download(f'{path}/{file.name + i})')
                                            #flywheel.api.containers_api.ContainersApi.get_container_file_zip_info(container_id= acquisition.id, file_name = file.name, member = file_list[i])
                                    #If the file exists, create a duplicate file with a different name.
#OSError: [Errno 22] Invalid argument: 'C:\\Users\\Charm\\projects\\test_folder/Bikson/Unsorted/MB_ARM4/unknown/<MPR Collection>/1.3.12.2.1107.5.2.43.166037.30000024041713434353700000020.dicom.zip'